# 05-xrefs
Get xrefs from a variety of sources
- Drugs: 
UMLS has mesh xrefs. From mesh, we can get UNII and CAS. From UNII_FDA, we can get inchikeys 
(lookup using cas or unii). From chembl, we can get chembl IDs from the inchikeys
So: UMLS -> mesh -> unii/cas -> inchikey -> chembl
insane, I know.
- Anatomy: uberon has umls xrefs
- disease: DO has umls, umls has NCI, ICD10PCS, SNOMEDCT_US, ICD10CM, OMIM
- proteins: umls has uniprot xrefs
- biological_process_or_activity/activity_and_behavior: umls has GO
- gene: umls has HGNC and OMIM

In [1]:
import sys
import os
import pickle
%matplotlib inline
import pandas as pd
from numpy import nan
import seaborn as sns
import shelve
import re
from collections import defaultdict, Counter
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from itertools import chain
from more_itertools import chunked
from collections import Counter
from pprint import pprint
import requests
from pyquery import PyQuery as pq
from wikidataintegrator import wdi_helpers, wdi_core, wdi_login
from semmed_biolink_environment import *

In [2]:
uri_to_curie = lambda s: s.split("/")[-1].replace("_", ":")

In [3]:
nodes = pd.read_csv(NODES_BIOLINK_TSV, sep='\t', index_col=0)

In [4]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_category
ID,,,,
C0270951,Ocular muscular dystrophy,T047,Disease or Syndrome,DiseaseOrPhenotypicFeature
C1706324,EREG wt Allele,T028,Gene or Genome,Gene
C0075561,sulfochloranthine,T109|T121,Organic Chemical|Pharmacologic Substance,ChemicalSubstance
C3470895,LINC00607 gene,T028,Gene or Genome,Gene
C0486433,Extended-spectrum beta lactamase,T126|T116,"Amino Acid, Peptide, or Protein|Enzyme",Protein


In [5]:
nodes.blm_category.value_counts()

ChemicalSubstance              58700
DiseaseOrPhenotypicFeature     36958
Protein                        28019
Gene                           23788
GrossAnatomicalStructure        9086
BiologicalProcessOrActivity     7665
AnatomicalEntity                2757
CellularComponent               1732
Cell                            1255
ActivityAndBehavior              881
PhenotypicFeature                418
GenomicEntity                    172
Name: blm_category, dtype: int64

## parse UMLS flat file to get all UMLS xrefs
# see: https://www.ncbi.nlm.nih.gov/books/NBK9685/

In [6]:
names = "CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X".split(",")
umls = pd.read_csv(MRCONSO_ENG_ARCHIVE, delimiter="|", names=names, index_col=None)
# only get CUIs in our list of nodes
umls = umls[umls.CUI.isin(nodes.index)]

/Users/richard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
umls['xref'] = umls.SAB + ":" + umls.CODE.map(str)
# easy fix to HGNC prefix duplication between SAB and CODE
umls.xref = umls.xref.str.replace("HGNC:HGNC:", "HGNC:")
# fix this MSH MESH nonsense
umls.xref = umls.xref.str.replace("MSH:", "MESH:")
# NCI_FDA is UNII
umls.xref = umls.xref.str.replace("NCI_FDA:", "UNII:")

In [8]:
umls.head(10)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X,xref
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,RXNORM:1926948
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,MTH:NOCODE
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,NaN,M0023172,D015060,MSH,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",0,N,NaN,NaN,MESH:D015060
5,C0000039,ENG,P,L0000039,VC,S1357296,Y,A1317708,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dipalmitoylphosphatidylcholine",0,N,NaN,NaN,MESH:D015060
6,C0000039,ENG,S,L0000035,PF,S0007560,Y,A26674543,NaN,M0023172,D015060,MSH,ET,D015060,"1,2-Dihexadecyl-sn-Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
7,C0000039,ENG,S,L0000035,VO,S1357276,Y,A1317687,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dihexadecyl sn Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
8,C0000039,ENG,S,L0000038,PF,S0007563,Y,A26661070,NaN,M0023172,D015060,MSH,ET,D015060,"1,2-Dipalmitoyl-Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
9,C0000039,ENG,S,L0000038,VO,S1357295,Y,A1317707,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dipalmitoyl Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
10,C0000039,ENG,S,L0012507,PF,S0033298,N,A18399186,NaN,LP15542-1,NaN,LNC,LPN,LP15542-1,Dipalmitoylphosphatidylcholine,0,N,256.0,NaN,LNC:LP15542-1
11,C0000039,ENG,S,L0012507,PF,S0033298,N,A22817493,166113012.0,102735002,NaN,SNOMEDCT_US,OAP,102735002,Dipalmitoylphosphatidylcholine,9,O,256.0,NaN,SNOMEDCT_US:102735002


In [9]:
XREF = dict(umls.groupby("CUI")['xref'].apply(set))
XREF = defaultdict(set, XREF)
print(XREF['C0000039'])

{'LNC:LP15542-1', 'MESH:D015060', 'RXNORM:1926948', 'LNC:MTHU010538', 'SNOMEDCT_US:102735002', 'MTH:NOCODE'}


### Chemicals and drugs

In [10]:
# what xrefs are on chemicals?
chem_umls = nodes[nodes.blm_category == "ChemicalSubstance"].index
xref_chem = {k:v for k,v in XREF.items() if k in chem_umls}
print(len(chem_umls))
c = Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in xref_chem.values()])))
pprint(c.most_common(25))
# nearly all have a mesh ID. not much of anything else
# neither mesh nor umls have inchikeys, or inchi, or smiles or anything useful for linking out
# blech

58700
[('MESH', 51182),
 ('SNOMEDCT_US', 15435),
 ('NCI', 11047),
 ('RXNORM', 9143),
 ('CHV', 8331),
 ('MMSL', 8046),
 ('MTH', 7294),
 ('MTHSPL', 6649),
 ('UNII', 6573),
 ('NDDF', 6371),
 ('LNC', 5558),
 ('VANDF', 4040),
 ('DRUGBANK', 3496),
 ('GS', 3406),
 ('ATC', 3204),
 ('PDQ', 2965),
 ('MEDCIN', 2914),
 ('CSP', 2896),
 ('USP', 1887),
 ('LCH_NW', 1826),
 ('NCI_NCI-GLOSS', 1505),
 ('NCI_CTRP', 1007),
 ('USPMG', 939),
 ('NCI_DTP', 834),
 ('NCI_DCP', 734)]


In [11]:
pd.set_option("display.width", 120)

URL = "http://id.nlm.nih.gov/mesh/sparql"
PREFIX = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
"""

def sparql_query(query):
    params = {'query': PREFIX + query, 'format': 'JSON', 'limit': 1000, 'offset': 0}
    r = requests.get(URL, params=params)
    res = [{k: v['value'] for k, v in x.items()} for x in r.json()['results']['bindings']]
    t = tqdm()
    while True:
        t.update(1)
        params['offset'] += 1000
        r = requests.get(URL, params=params).json()['results']['bindings']
        if not r:
            break
        res.extend([{k: v['value'] for k, v in x.items()} for x in r])
    df = pd.DataFrame(res)
    return df

In [12]:
query = """
SELECT distinct ?mesh ?meshLabel ?r ?rr
FROM <http://id.nlm.nih.gov/mesh> WHERE {
  ?mesh meshv:active 1 .
  ?mesh meshv:preferredMappedTo ?p .
  ?p meshv:treeNumber ?treeNum .
  FILTER(STRSTARTS(STR(?treeNum), "http://id.nlm.nih.gov/mesh/D")) .
  ?mesh rdfs:label ?meshLabel .
  ?mesh meshv:preferredConcept [meshv:registryNumber ?r] .
  #OPTIONAL {?mesh meshv:preferredConcept [meshv:relatedRegistryNumber ?rr]}
}
"""
df = sparql_query(query)

218it [05:46,  1.84s/it]

In [13]:
df.r = df.r.replace("0", nan)
df.dropna(subset=["r"], inplace=True)
df = df[~df.r.str.startswith("EC ")]
df.mesh = df.mesh.str.replace("http://id.nlm.nih.gov/mesh/", "")
df.set_index("mesh", inplace=True)

In [14]:
df.to_csv(MESH_XREFS_TSV, sep='\t')
df.head()

,meshLabel,r
mesh,,
C000603935,aripiprazole lauroxil,B786J7A343
C487637,telavancin,XK134822Z0
C070576,"PML protein, human",143220-95-5
C100708,oritavancin,PUG62FRZ2E
C000603533,3-methyl-3-buten-1-ol,KJ25C8CPFA


In [15]:
mesh_xrefs = pd.read_csv(MESH_XREFS_TSV, sep='\t', index_col=0)
mesh_xrefs.r = mesh_xrefs.r.apply(lambda x: "CAS:" + x if "-" in x else "UNII:" + x)
mesh_xrefs = mesh_xrefs.groupby("mesh").r.apply(set).to_dict()
mesh_xrefs = {"MESH:"+k:v for k,v in mesh_xrefs.items()}
len(mesh_xrefs)

39652

In [16]:
for k,v in xref_chem.items():
    for vv in list(v):
        if vv in mesh_xrefs:
            v.update(mesh_xrefs[vv])

In [17]:
# download: 'http://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip'
unii_df = pd.read_csv(UNII_RECORDS, dtype=str, sep='\t', low_memory=False)
unii_df.dropna(subset=['INCHIKEY'], inplace=True)

In [18]:
unii_df.head()

,UNII,PT,RN,EC,NCIT,RXCUI,PUBCHEM,ITIS,NCBI,PLANTS,GRIN,MPNS,INN_ID,MF,INCHIKEY,SMILES,INGREDIENT_TYPE
0,00174624E2,CHF-6333 CATION,1613620-10-2,NaN,NaN,NaN,76285164,NaN,NaN,NaN,NaN,NaN,NaN,C27H28F3N6O3,IHTRPSMRGYWUIM-HSZRJFAPSA-O,COC(=O)C1=C(C)N(C2=NNC(=O)N2[C@@H]1C3=CC=C(C=C...,IONIC MOIETY
1,0129526470,"5,8-DIMETHOXY(1,2,4)TRIAZOLO(1,5-C)PYRIMIDIN-2...",219715-62-5,NaN,NaN,NaN,11446888,NaN,NaN,NaN,NaN,NaN,NaN,C7H9N5O2,DBJPBHJHAPAUQU-UHFFFAOYSA-N,COC1=CN=C(OC)N2N=C(N)N=C12,INGREDIENT SUBSTANCE
3,0377415922,"N-DESMETHYLVENLAFAXINE, (S)-",392332-59-1,NaN,NaN,NaN,9860056,NaN,NaN,NaN,NaN,NaN,NaN,C16H25NO2,MKAFOJAJJMUXLW-OAHLLOKOSA-N,CNC[C@H](C1=CC=C(OC)C=C1)C2(O)CCCCC2,INGREDIENT SUBSTANCE
5,0480546720,HOMOCYCLOLEUCINE HYDROCHLORIDE,39692-17-6,254-594-3,NaN,NaN,2724466,NaN,NaN,NaN,NaN,NaN,NaN,C7H13NO2.ClH,GTKXSYHXQSKWNP-UHFFFAOYSA-N,Cl.NC1(CCCCC1)C(O)=O,INGREDIENT SUBSTANCE
6,0503177591,SUCCINALDEHYDE,638-37-9,211-333-8,NaN,NaN,12524,NaN,NaN,NaN,NaN,NaN,NaN,C4H6O2,PCSMJKASWLYICJ-UHFFFAOYSA-N,O=CCCC=O,INGREDIENT SUBSTANCE


In [19]:
n=0
for k,v in tqdm_notebook(xref_chem.items()):
    for vv in list(v):
        if vv.startswith("UNII:"):
            xref = vv.replace("UNII:", "")
            s = unii_df.query("UNII == @xref").INCHIKEY
            if not s.empty:
                n+=1
                v.add("INCHIKEY:" + list(s)[0])

In [20]:
xref_inchi = {k:v for k,v in xref_chem.items() if any(vv.startswith("INCHIKEY:") for vv in v)}
xref_inchi = {k:[vv for vv in v if vv.startswith("INCHIKEY:")][0].replace("INCHIKEY:", "") for k,v in xref_inchi.items()}
print(len(xref_inchi))
list(xref_inchi.items())[:4]

10290


[('C0000248', 'CZIHNRWJTSTCEX-UHFFFAOYSA-N'),
 ('C0000294', 'XOGTZOOQQBDUSI-UHFFFAOYSA-M'),
 ('C0000378', 'QXWYKJLNLSIPIN-JGVFFNPUSA-N'),
 ('C0000399', 'CBOKZNLSFMZJJA-PEBGCTIMSA-N')]

In [22]:
url = "https://www.ebi.ac.uk/chembl/api/data/molecule?molecule_structures__standard_inchi_key__in={}&format=json&limit=100"
for chunk in tqdm(chunked(xref_inchi.items(), 100), total=len(xref_inchi)/100):
    chunk = dict(chunk)
    chunk = {v:k for k,v in chunk.items()}
    inchis = ",".join(chunk)
    mols = requests.get(url.format(inchis)).json()['molecules']
    for m in mols:
        chembl = m['molecule_chembl_id']
        inchi = m['molecule_structures']['standard_inchi_key']
        XREF[chunk[inchi]].add("CHEMBL:" + chembl)


  0%|          | 0/102.9 [00:00<?, ?it/s]
  1%|          | 1/102.9 [00:07<12:53,  7.59s/it]
  2%|▏         | 2/102.9 [00:11<10:58,  6.53s/it]
  3%|▎         | 3/102.9 [00:15<09:44,  5.85s/it]
  4%|▍         | 4/102.9 [00:19<08:26,  5.12s/it]
  5%|▍         | 5/102.9 [00:23<07:46,  4.77s/it]
  6%|▌         | 6/102.9 [00:26<07:00,  4.34s/it]
  7%|▋         | 7/102.9 [00:29<06:24,  4.01s/it]
  8%|▊         | 8/102.9 [00:33<05:56,  3.76s/it]
  9%|▊         | 9/102.9 [00:36<05:40,  3.63s/it]
 10%|▉         | 10/102.9 [00:39<05:35,  3.61s/it]
 11%|█         | 11/102.9 [00:43<05:21,  3.50s/it]
 12%|█▏        | 12/102.9 [00:46<05:16,  3.48s/it]
 13%|█▎        | 13/102.9 [00:49<05:06,  3.41s/it]
 14%|█▎        | 14/102.9 [00:55<06:06,  4.13s/it]
 15%|█▍        | 15/102.9 [00:58<05:41,  3.88s/it]
 16%|█▌        | 16/102.9 [01:02<05:19,  3.67s/it]
 17%|█▋        | 17/102.9 [01:05<04:58,  3.48s/it]
 17%|█▋        | 18/102.9 [01:08<04:47,  3.39s/it]
 18%|█▊        | 19/102.9 [01:11<04:47,  3.42s/i

In [23]:
len({k:v for k,v in XREF.items() if any(vv.startswith("CHEMBL:") for vv in v)})

8088

In [24]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

## UBERON, uses uberon.csv generated from the uberon.owl file previously downloaded (see README)

In [25]:
df = pd.read_csv(UBERON_CSV)
df = df[df.xref.str.startswith("UMLS:")]
df.xref = df.xref.str.replace("UMLS:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
9755,UBERON:0007227,C0228757
11788,UBERON:0002965,C0175263
11853,UBERON:0001532,C0007276
11897,UBERON:0014636,C0228627
11904,UBERON:0002610,C0152411


In [26]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [27]:
XREF['C1272528']

{'MTH:NOCODE', 'SNOMEDCT_US:384765009', 'UBERON:0006472'}

## DOID, uses doid.csv generated from the doi.owl file previously downloaded (see README)

In [28]:
df = pd.read_csv(DOID_CSV)
df.dropna(inplace=True)
df = df[df.xref.str.startswith("UMLS_CUI:")]
df.xref = df.xref.str.replace("UMLS_CUI:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
6,DOID:3864,C1332188
7,DOID:3864,C0278876
21,DOID:699,C0162670
27,DOID:0111831,C1844936
34,DOID:0111618,C1853116


In [29]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [30]:
XREF['C0263518']

{'DOID:1943',
 'HPO:HP:0025470',
 'ICD10:L65.0',
 'ICD10AM:L65.0',
 'ICD10CM:L65.0',
 'ICD9CM:704.02',
 'ICPC2ICD10ENG:MTHU025078',
 'ICPC2ICD10ENG:MTHU073653',
 'MDR:10043200',
 'MEDCIN:37106',
 'MTH:NOCODE',
 'NCI:C112200',
 'NCI_NICHD:C112200',
 'SNOMEDCT_US:201147004',
 'SNOMEDCT_US:39479004'}

In [31]:
XREF['C0591520']

{'CHV:0000041179', 'MESH:D000068298'}

## proteins

In [32]:
# See README regarding MRSAT_ARCHIVE
names = list("abcdefghijklmn")
iter_csv = pd.read_csv(MRSAT_ARCHIVE, delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_uniprot = dict()
for chunk in tqdm(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "SWISS_PROT"]
    d = dict(zip(chunk.a, chunk.k))
    umls_uniprot.update(d)


  0%|          | 0/67.668372 [00:00<?, ?it/s]
  1%|▏         | 1/67.668372 [00:03<03:23,  3.05s/it]/Users/richard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2,5,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):

  3%|▎         | 2/67.668372 [00:06<03:20,  3.05s/it]
  4%|▍         | 3/67.668372 [00:09<03:20,  3.10s/it]/Users/richard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):

  6%|▌         | 4/67.668372 [00:12<03:15,  3.07s/it]
  7%|▋         | 5/67.668372 [00:15<03:15,  3.11s/it]
  9%|▉         | 6/67.668372 [00:18<03:08,  3.06s/it]
 10%|█         | 7/67.668372 [00:21<03:03,  3.03s/it]
 12%|█▏        | 8/67.668372 [00:24<03:02,  3.05s/it]
 13%|█

In [33]:
len(umls_uniprot)

4916

In [34]:
for umls, uniprot in umls_uniprot.items():
    XREF[umls].add("UNIPROT:" + uniprot)

In [35]:
XREF['C0215993']

{'MESH:C081092',
 'MTH:NOCODE',
 'NCI:C127008',
 'NCI_CTRP:C127008',
 'UNIPROT:Q04756'}

In [36]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

In [37]:
nodes['xrefs'] = nodes.index.map(lambda x: ";".join(XREF.get(x,list())))

In [38]:
nodes.head(20)

,LABEL,umls_type,umls_type_label,blm_category,xrefs
ID,,,,,
C0270951,Ocular muscular dystrophy,T047,Disease or Syndrome,DiseaseOrPhenotypicFeature,ICPC2ICD10ENG:MTHU051186;ICPC2ICD10ENG:MTHU053...
C1706324,EREG wt Allele,T028,Gene or Genome,Gene,MTH:NOCODE;NCI:C51422
C0075561,sulfochloranthine,T109|T121,Organic Chemical|Pharmacologic Substance,ChemicalSubstance,CAS:55945-60-3;MESH:C022876
C3470895,LINC00607 gene,T028,Gene or Genome,Gene,MTH:NOCODE;HGNC:43944
C0486433,Extended-spectrum beta lactamase,T126|T116,"Amino Acid, Peptide, or Protein|Enzyme",Protein,LNC:LP17264-0;CHV:0000037238;SNOMEDCT_US:10276...
C0393778,Congenital anosmia,T019,Congenital Abnormality,DiseaseOrPhenotypicFeature,OMIM:301700;SNOMEDCT_US:230502003;OMIM:107200;...
C0543506,clinical acidosis,T046,Pathologic Function,DiseaseOrPhenotypicFeature,CHV:0000038669
C0596286,Cell Growth Regulation,T043,Cell Function,BiologicalProcessOrActivity,CSP:0602-2830;GO:GO:0001558;NCI:C18080
C3273836,MIA wt Allele,T028,Gene or Genome,Gene,MTH:NOCODE;NCI_CTRP:C98125;NCI:C98125


In [39]:
nodes.to_csv(NODES_XREF_TSV, sep='\t')